# Thu Apr 27 08:43:45 PM CEST 2023

fr. `exploration.ipynb`: _Working with EdgeGPT in Jupyter_

Reason for the funny structure of this document is that I want to have my `await` cell at the bottom so that if it has an error it does not scroll out other cells out of the page. I further more have the _Working example_ below the _Chat_ section for the case that _Working example_ would be unfolded, in which case that too would scroll information out of the page.

## Chat

In [3]:
# Import the reverse engineered API.
from EdgeGPT import Chatbot, ConversationStyle

# Set cookies
cookies_ai0 = r"./cookies/cookies_ai0.json"
cookies_ai1 = r"./cookies/cookies_ai1.json"
cookies_ai2 = r"./cookies/cookies_ai2.json"
cookies_ai3 = r"./cookies/cookies_ai3.json"
cookies_ai4 = r"./cookies/cookies_ai1.json"

# Import the os module for operating system functions
import os

# Define a function to create directories if they do not exist
def create_dirs(*dirs):
    """Create directories if they do not exist.

    Args:
        *dirs: One or more directory names as strings.

    Prints:
        A message for each directory that is created.
    """
    # Loop through the directories
    for dir in dirs:
        # Check if the directory exists
        if not os.path.exists(dir):
            # If not, create the directory
            os.mkdir(dir)
            # Print a message
            print(f"Created {dir}")

# Call the function with the names of the directories as arguments.
create_dirs("chats", "chats_AdaptiveCards")


Created chats_AdaptiveCards


In [ ]:
bot.reset() # Run when needed

<coroutine object Chatbot.reset at 0x00000256B1A47E40>

In [ ]:
bot.close() # Run when needed

<coroutine object Chatbot.close at 0x7f81b05c54d0>

In [ ]:
bot = Chatbot(cookie_path=cookies_ai1) # Run when needed

# Define a function to create two new files that log the chat conversation
def create_chat_log_files():
    """Create two new files that log the chat conversation with the format chat<number>.md and chat<number>_adaptiveCard.md.

    The number is defined by the global variable chat_session_count.
    The files are created in the chat and chat_adaptiveCards directories respectively and are empty.
    """
    # Use the global keyword to access and modify the chat_session_count variable
    global chat_session_count

    # Get a list of all the files in the chat directory
    chat_files = os.listdir("chat")

    # If the list is empty, set chat_session_count to zero
    if not chat_files:
        chat_session_count = 0
    # Otherwise, find the file with the highest number in its name
    else:
        # Initialize a variable to store the highest number found
        highest_number = 0
        # Loop through the list of files
        for file in chat_files:
            # Extract the number from the file name using string slicing and int()
            number = int(file[4:-3])
            # Compare it with the highest number and update it if necessary
            if number > highest_number:
                highest_number = number
        # Set chat_session_count to one more than the highest number found
        chat_session_count = highest_number + 1

    # Create the file names using f-strings
    chat_log_file = f"chat{chat_session_count}.md"
    adaptive_card_file = f"chat{chat_session_count}_adaptiveCard.md"

    # Join the directory names with the file names using os.path.join()
    chat_log_path = os.path.join("chat", chat_log_file)
    adaptive_card_path = os.path.join("chat_adaptiveCards", adaptive_card_file)

    # Open the files in write mode and close them
    with open(chat_log_path, "w") as clf, open(adaptive_card_path, "w") as acf:
        clf.close()
        acf.close()

    # Increment the chat_session_count by 1
    chat_session_count += 1

# Call the function to create the first pair of files
create_chat_log_files()


In [ ]:
# resp['item']['messages'][1]['text']
resp["item"]["messages"][1]["adaptiveCards"][0]["body"][0]["text"]

"Okay, let's move on to the second part of the code: defining the hyperparameters and the dataset. Hyperparameters are variables that control the training process and the architecture of the neural network. We will use the values that you specified in your request, such as the number of epochs, the learning rate, the batch size, and the number of neurons in each layer. We will also define some constants, such as the adiabatic index Γ and the dimensionality of the input and output.\n\nTo create the dataset, we will use the equations that you provided to generate random samples of primitive and conservative variables. We will use NumPy to perform the calculations and then convert the arrays to PyTorch tensors. We will also split the dataset into training and testing subsets, using 80% for training and 20% for testing. We will use torch.utils.data.TensorDataset and torch.utils.data.DataLoader to create PyTorch datasets and data loaders that will allow us to iterate over the data in batche

In [ ]:
prompt = '''
I will write to you a request to generate code in PyTorch for a neural network. 
I will explain what you should generate at great length, but please use the 
paper Machine Learning for Conservative-to-Primitive in Relativistic 
Hydrodynamics by Dieseldorst et al. for anything that I miss to specify, as I 
have based my description on this paper. It would be extremely helpful to me if 
you could generate this code. I will write my request in batches, please only 
start generating code when I have completed my request fully. I will mark that I 
have completed my request by saying "This completes my request". I will start 
writing my request now.

Please create a code in PyTorch for conservative-to-primitive inversion based on 
supervised learning of a fully connected feedforward neural network. Use the GPU 
if it is available. Use for the neural network two hidden layers and use the 
sigmoid function as the activation function for each of the two. Use ReLU as the 
nonlinearity applied to the output. Use for the number of neurons in the first 
hidden layer 600, and for the number of neurons of the second hidden layer 200. 
The network has three inputs, the conserved density D, the conserved momentum in 
the x direction S_x and the conserved energy density τ. The network has one 
output, namely the primitive variable which is the pressure p. All other 
primitive variables can be calculated from p. 

Let the number of epochs be 400. The training dataset should consist of 80000 
samples, while the test dataset should consist of 10000 samples. Let the initial 
learning rate for training be set to 6 * 10^-4. Please construct the training 
dataset as follows. Use equation (3) from Dieseldorst et al., p = p(ρ,e) = 
(Γ-1)ρe, to calculate the pressure from the equation of state. Then uniformly 
sample the primitive variables over the intervals ρ ∈ (0,10.1), ϵ ∈ (0, 2.02), 
and v_x ∈ (0, 0.721) using a random seed and by calculate the corresponding 
conservative variables D, S_x, and τ, using the equations (2) from Dieseldorst 
et al.: W = (1-v_x^2)^(-1/2), h = 1 + ϵ 
+ p / ρ, D = ρ W, S_x = ρ h W^2 v_x, τ = ρ h W^2 - p - D.

Adapt the learning rate until the error on the training dataset is minimized, 
which marks that training is completed. To adapt the learning rate, we multiply 
the learning rate by a factor of 0.5 whenever the loss of the training data over 
the last five epochs has not improved by at least 0.05% with respect to the 
previous five epochs. Furthermore, ten epochs have to be completed before the 
next possible learning rate adaption. Use torch's ReduceLROnPlateau for the 
learning rate adaptation.

Errors on data series should be evaluated with the L_1 and L_{infinity} norms. 
Errors are calculated by comparing the error in the calculated pressure after 
the trained neural network performs the conservative to primitive inversion and 
by comparing to the test dataset.

The minimization of the weights and biases, collectively called θ, should be 
performed iteratively, by 1. Computing the loss function E, for which we use the 
mean squared error, and 2. Taking the gradient of the loss function with 
backpropagation, and 3. Applying the gradient descent algorithm, for which we 
use the Adam optimizer, to minimize E. An epoch is completed by performing these 
three steps for all samples of the training dataset. Let the training dataset be 
split into random mini-batches of size 32 and collect the gradients of the θ for 
all samples of a minibatch. Apply the gradient descent algorithm after each 
mini-batch. Create new mini-batches after each epoch.

We use the pressure to calculate all other primitive variables, using equations 
(A2), (A3),(A4), (A5), from Dieseldorst et al. Using these equations, we 
calculate the primitive velocity in the x-direction to be v_x(p) =  S_x / (τ + D 
+ p), we calculate the Lorentz factor to be W(p) = 1 / (1- v^2(p))^(1/2), we 
calculate the primitive variable specific internal energy to be ϵ(p) = (τ + D(1- 
W(p) + p(1- W^2(p)) / (D W(p)) and we calculate the primitive variable density 
to be ρ(p) = D / (W(p)).

The code should print the progress and should plot all the relevant results. 
Make sure that in plotting no errors are thrown due to mixing of numpy arrays 
and torch tensors, and to that end convert all numpy arrays to torch tensors. 
Create, among other plots, the mean squared error against epochs for the 
training data, the testing data and the learning adaptation, both as separate 
plots as well as all in one. Furthermore, create a plot of the learning rate 
against epoch. Use different color palettes for the plots to make them more 
appealing and distinguishable. Add labels, legends, titles, and annotations to 
the plots to make them more informative and clear. Adjust the figure size, 
aspect ratio, margins, and spacing of the plots to make them more readable. Use 
different kinds of plots or combine multiple plots to show different aspects of 
the data or highlight interesting patterns. 

The code should save all results and plots. Please explain in great detail the 
code step-by-step in the code comments. Make the code readable by creating many 
functions to perform the tasks that I have described.

This completes my request.
'''

In [ ]:
prompt = 'I am more than ready!'
prompt = 'No questions or comments, please continue'

In [ ]:
# resp['item']['messages'][1]['text']
resp["item"]["messages"][1]["adaptiveCards"][0]["body"][0]["text"]

"Okay, let's move on to the second part of the code: defining the hyperparameters and the dataset. Hyperparameters are variables that control the training process and the architecture of the neural network. We will use the values that you specified in your request, such as the number of epochs, the learning rate, the batch size, and the number of neurons in each layer. We will also define some constants, such as the adiabatic index Γ and the dimensionality of the input and output.\n\nTo create the dataset, we will use the equations that you provided to generate random samples of primitive and conservative variables. We will use NumPy to perform the calculations and then convert the arrays to PyTorch tensors. We will also split the dataset into training and testing subsets, using 80% for training and 20% for testing. We will use torch.utils.data.TensorDataset and torch.utils.data.DataLoader to create PyTorch datasets and data loaders that will allow us to iterate over the data in batche

In [ ]:
resp = await bot.ask(prompt=prompt, conversation_style=ConversationStyle.creative, wss_link="wss://sydney.bing.com/sydney/ChatHub")
# resp["item"]["messages"][1]["adaptiveCards"][0]["body"][0]["text"]
resp['item']['messages'][1]['text']

"Alright, let's proceed to the third part of the code: defining the neural network model and the loss function. We will use PyTorch's nn.Module class to create a custom model that consists of three fully connected layers with ReLU activation functions. The first layer will take the input vector of size num_input and output a hidden vector of size num_hidden1. The second layer will take the hidden vector of size num_hidden1 and output another hidden vector of size num_hidden2. The third layer will take the hidden vector of size num_hidden2 and output a prediction vector of size num_output. We will also use nn.init.kaiming_uniform_ to initialize the weights of the layers according to the He initialization method.\n\nTo define the loss function, we will use PyTorch's nn.MSELoss class to create a mean squared error (MSE) criterion that measures the difference between the predicted pressure and the true pressure. We will also use torch.sqrt to calculate the root mean squared error (RMSE) fo

## Working example with Bing description 7 prompt

In [ ]:
from EdgeGPT import Chatbot, ConversationStyle
cookies_ai0 = r"./cookies/cookies_ai0.json"
cookies_ai1 = r"./cookies/cookies_ai1.json"
cookies_ai2 = r"./cookies/cookies_ai2.json"
cookies_ai3 = r"./cookies/cookies_ai3.json"
cookies_ai4 = r"./cookies/cookies_ai1.json"

In [ ]:
bot.reset() # Run when needed

<coroutine object Chatbot.reset at 0x00000256B1A47E40>

In [ ]:
bot.close() # Run when needed

<coroutine object Chatbot.close at 0x7f81b05c54d0>

In [ ]:
bot = Chatbot(cookie_path=cookies_ai1) # Run when needed

In [ ]:
# resp['item']['messages'][1]['text']
resp["item"]["messages"][1]["adaptiveCards"][0]["body"][0]["text"]

"Okay, let's move on to the second part of the code: defining the hyperparameters and the dataset. Hyperparameters are variables that control the training process and the architecture of the neural network. We will use the values that you specified in your request, such as the number of epochs, the learning rate, the batch size, and the number of neurons in each layer. We will also define some constants, such as the adiabatic index Γ and the dimensionality of the input and output.\n\nTo create the dataset, we will use the equations that you provided to generate random samples of primitive and conservative variables. We will use NumPy to perform the calculations and then convert the arrays to PyTorch tensors. We will also split the dataset into training and testing subsets, using 80% for training and 20% for testing. We will use torch.utils.data.TensorDataset and torch.utils.data.DataLoader to create PyTorch datasets and data loaders that will allow us to iterate over the data in batche

In [ ]:
prompt = '''
I will write to you a request to generate code in PyTorch for a neural network. 
I will explain what you should generate at great length, but please use the 
paper Machine Learning for Conservative-to-Primitive in Relativistic 
Hydrodynamics by Dieseldorst et al. for anything that I miss to specify, as I 
have based my description on this paper. It would be extremely helpful to me if 
you could generate this code. I will write my request in batches, please only 
start generating code when I have completed my request fully. I will mark that I 
have completed my request by saying "This completes my request". I will start 
writing my request now.

Please create a code in PyTorch for conservative-to-primitive inversion based on 
supervised learning of a fully connected feedforward neural network. Use the GPU 
if it is available. Use for the neural network two hidden layers and use the 
sigmoid function as the activation function for each of the two. Use ReLU as the 
nonlinearity applied to the output. Use for the number of neurons in the first 
hidden layer 600, and for the number of neurons of the second hidden layer 200. 
The network has three inputs, the conserved density D, the conserved momentum in 
the x direction S_x and the conserved energy density τ. The network has one 
output, namely the primitive variable which is the pressure p. All other 
primitive variables can be calculated from p. 

Let the number of epochs be 400. The training dataset should consist of 80000 
samples, while the test dataset should consist of 10000 samples. Let the initial 
learning rate for training be set to 6 * 10^-4. Please construct the training 
dataset as follows. Use equation (3) from Dieseldorst et al., p = p(ρ,e) = 
(Γ-1)ρe, to calculate the pressure from the equation of state. Then uniformly 
sample the primitive variables over the intervals ρ ∈ (0,10.1), ϵ ∈ (0, 2.02), 
and v_x ∈ (0, 0.721) using a random seed and by calculate the corresponding 
conservative variables D, S_x, and τ, using the equations (2) from Dieseldorst 
et al.: W = (1-v_x^2)^(-1/2), h = 1 + ϵ 
+ p / ρ, D = ρ W, S_x = ρ h W^2 v_x, τ = ρ h W^2 - p - D.

Adapt the learning rate until the error on the training dataset is minimized, 
which marks that training is completed. To adapt the learning rate, we multiply 
the learning rate by a factor of 0.5 whenever the loss of the training data over 
the last five epochs has not improved by at least 0.05% with respect to the 
previous five epochs. Furthermore, ten epochs have to be completed before the 
next possible learning rate adaption. Use torch's ReduceLROnPlateau for the 
learning rate adaptation.

Errors on data series should be evaluated with the L_1 and L_{infinity} norms. 
Errors are calculated by comparing the error in the calculated pressure after 
the trained neural network performs the conservative to primitive inversion and 
by comparing to the test dataset.

The minimization of the weights and biases, collectively called θ, should be 
performed iteratively, by 1. Computing the loss function E, for which we use the 
mean squared error, and 2. Taking the gradient of the loss function with 
backpropagation, and 3. Applying the gradient descent algorithm, for which we 
use the Adam optimizer, to minimize E. An epoch is completed by performing these 
three steps for all samples of the training dataset. Let the training dataset be 
split into random mini-batches of size 32 and collect the gradients of the θ for 
all samples of a minibatch. Apply the gradient descent algorithm after each 
mini-batch. Create new mini-batches after each epoch.

We use the pressure to calculate all other primitive variables, using equations 
(A2), (A3),(A4), (A5), from Dieseldorst et al. Using these equations, we 
calculate the primitive velocity in the x-direction to be v_x(p) =  S_x / (τ + D 
+ p), we calculate the Lorentz factor to be W(p) = 1 / (1- v^2(p))^(1/2), we 
calculate the primitive variable specific internal energy to be ϵ(p) = (τ + D(1- 
W(p) + p(1- W^2(p)) / (D W(p)) and we calculate the primitive variable density 
to be ρ(p) = D / (W(p)).

The code should print the progress and should plot all the relevant results. 
Make sure that in plotting no errors are thrown due to mixing of numpy arrays 
and torch tensors, and to that end convert all numpy arrays to torch tensors. 
Create, among other plots, the mean squared error against epochs for the 
training data, the testing data and the learning adaptation, both as separate 
plots as well as all in one. Furthermore, create a plot of the learning rate 
against epoch. Use different color palettes for the plots to make them more 
appealing and distinguishable. Add labels, legends, titles, and annotations to 
the plots to make them more informative and clear. Adjust the figure size, 
aspect ratio, margins, and spacing of the plots to make them more readable. Use 
different kinds of plots or combine multiple plots to show different aspects of 
the data or highlight interesting patterns. 

The code should save all results and plots. Please explain in great detail the 
code step-by-step in the code comments. Make the code readable by creating many 
functions to perform the tasks that I have described.

This completes my request.
'''

In [ ]:
%%script echo skipping
prompt = 'I am more than ready!'
prompt = 'No questions or comments, please continue'

In [ ]:
# resp['item']['messages'][1]['text']
resp["item"]["messages"][1]["adaptiveCards"][0]["body"][0]["text"]

"Okay, let's move on to the second part of the code: defining the hyperparameters and the dataset. Hyperparameters are variables that control the training process and the architecture of the neural network. We will use the values that you specified in your request, such as the number of epochs, the learning rate, the batch size, and the number of neurons in each layer. We will also define some constants, such as the adiabatic index Γ and the dimensionality of the input and output.\n\nTo create the dataset, we will use the equations that you provided to generate random samples of primitive and conservative variables. We will use NumPy to perform the calculations and then convert the arrays to PyTorch tensors. We will also split the dataset into training and testing subsets, using 80% for training and 20% for testing. We will use torch.utils.data.TensorDataset and torch.utils.data.DataLoader to create PyTorch datasets and data loaders that will allow us to iterate over the data in batche

In [ ]:
resp = await bot.ask(prompt=prompt, conversation_style=ConversationStyle.creative, wss_link="wss://sydney.bing.com/sydney/ChatHub")
# resp["item"]["messages"][1]["adaptiveCards"][0]["body"][0]["text"]
resp['item']['messages'][1]['text']

"Alright, let's proceed to the third part of the code: defining the neural network model and the loss function. We will use PyTorch's nn.Module class to create a custom model that consists of three fully connected layers with ReLU activation functions. The first layer will take the input vector of size num_input and output a hidden vector of size num_hidden1. The second layer will take the hidden vector of size num_hidden1 and output another hidden vector of size num_hidden2. The third layer will take the hidden vector of size num_hidden2 and output a prediction vector of size num_output. We will also use nn.init.kaiming_uniform_ to initialize the weights of the layers according to the He initialization method.\n\nTo define the loss function, we will use PyTorch's nn.MSELoss class to create a mean squared error (MSE) criterion that measures the difference between the predicted pressure and the true pressure. We will also use torch.sqrt to calculate the root mean squared error (RMSE) fo